In [5]:
import os
import torch
import rasterio
import numpy as np
from rasterio.warp import reproject, Resampling
from rasterio.transform import from_bounds
from glob import glob
import tifffile
# process one event files and convert to .pt files
# set up the path
hsv_base_path = "/home/sirui/INNOMAUS/2025_new/gt_b/r/"  #path for ground truth tif
dem_path = "/home/sirui/INNOMAUS/2025_new/dem/DEM.tif" #path for dem tif
runoff_base_path = "/home/sirui/INNOMAUS/2025_new/berlin_runoff" #path for runoff tif, the output from process.ipynb
output_path = "/home/sirui/INNOMAUS/2025_new/pt" #path for output .pt files
#format of the output file: [height, width, time, channels]
#channels: H, S, V, runoff, dem
os.makedirs(output_path, exist_ok=True)

def read_raster(file_path, target_meta=None):
    data=  tifffile.imread(file_path)
    """ read the raster file and reproject """
    with rasterio.open(file_path) as src:
        crs = src.crs
        meta = src.meta.copy()

        # process if crs is none
        if crs is None:
            print(f"Warning: {file_path} has no CRS. Assigning default EPSG:25833.")
            meta['crs'] = "EPSG:25833"  # projection

        if target_meta:
            target_transform = target_meta['transform']
            target_shape = (target_meta['height'], target_meta['width'])

            destination = np.zeros(target_shape, dtype=np.float32)
            reproject(
                source=data,
                destination=destination,
                src_transform=meta['transform'],
                src_crs=meta['crs'],
                dst_transform=target_transform,
                dst_crs=target_meta['crs'],
                resampling=Resampling.bilinear
            )
            return destination

        return data


# read dem
dem_data = read_raster(dem_path)
dem_data = np.nan_to_num(dem_data, nan=75.4477)
with rasterio.open(dem_path) as dem_src:
    dem_meta = dem_src.meta.copy()
    dem_meta.update({"width": 3212, "height": 2727})

# read data from events
event_folders = sorted(glob(os.path.join(hsv_base_path, "*/")))
for event_folder in event_folders:
    event_name = os.path.basename(os.path.dirname(event_folder))
    print(event_name)
    # read H, S, V files
    path_h = event_folder+ "*H.tif"
    path_u = event_folder+ "*U.tif"
    path_v = event_folder+ "*V.tif"
    h_files = sorted(glob(path_h))
    s_files = sorted(glob(path_u))
    v_files = sorted(glob(path_v))

    data_tensor = np.zeros((2727, 3212, 25, 5), dtype=np.float32)

    for h_file, s_file, v_file in zip(h_files, s_files, v_files):
        # read time index
        time_index = int(os.path.basename(h_file).split("_")[-1].replace("H.tif", ""))
        time_index = min(time_index // 300, 24)  # max limit
        
        # read H, S, V data
        h_data = read_raster(h_file, target_meta=dem_meta)
        s_data = read_raster(s_file, target_meta=dem_meta)
        v_data = read_raster(v_file, target_meta=dem_meta)

        # put into correct location
        data_tensor[:, :, time_index, 0] = h_data
        data_tensor[:, :, time_index, 1] = s_data
        data_tensor[:, :, time_index, 2] = v_data

    # read runoff data
    runoff_folder = os.path.join(runoff_base_path, "r", f"r_{event_name}")
    runoff_files = sorted(glob(os.path.join(runoff_folder, "*.tif")))
    for file in runoff_files:
        time_index = int(os.path.basename(file).split("_")[-1].replace(".tif", ""))
        time_index = min(time_index // 300, 24)
        runoff_data = read_raster(file, target_meta=dem_meta)
        print(runoff_data.max())
        data_tensor[:, :, time_index, 3] = runoff_data[:,:3212]

    # DEM data
    data_tensor[:, :, :, 4] = np.repeat(dem_data[:, :, np.newaxis], 25, axis=2)
    print(dem_data[10,10],"dem")
    # convert to torch format
    torch_tensor = torch.tensor(data_tensor)
    torch.save(torch_tensor, os.path.join(output_path, f"r_{event_name}.pt"))

print("done!")

100mm
7.8903174
13.553332
14.13009
16.82366
18.858234
20.339167
21.394156
12.096661
22.137583
22.659803
19.49212
14.824902
11.058647
8.3435335
6.991723
6.0047293
5.187728
4.5070353
13.288333
3.9358923
3.4532766
3.0426865
2.6911175
2.388262
13.513145
75.4477 dem
100y
3.2471077
6.6004553
6.6233797
6.6295657
6.6312323
7.469786
8.300561
5.212707
8.976067
9.516671
8.619169
7.0841084
5.663487
4.5116444
3.6065965
2.898262
2.3408623
2.0545542
6.2127953
1.8236837
1.6204839
1.4411566
1.2837964
1.147988
6.5160675
75.4477 dem
105mm
8.359084
14.232288
15.264409
18.055967
20.136341
21.632092
22.685783
12.781279
23.420826
23.93251
20.516947
15.514035
11.513381
8.652306
7.345034
6.2934484
5.4264092
4.706513
13.977247
4.1043115
3.5968292
3.1661313
2.798152
2.4817874
14.194684
75.4477 dem
10y
1.6817434
3.9969428
4.035208
4.048298
4.052757
4.0542736
4.0547895
2.800889
4.3469257
4.7634454
4.4713373
3.847133
3.2051215
2.6392622
2.1639132
1.771013
1.4475366
1.1808771
3.5783324
0.96029603
0.7995909
0.7124309